**Notebook explotar la información electoral de la elecciones generales**

Desde el Ministerio del Interior de España se pone a disposición de los ciudadanos la información relativa a las elecciones en 3 niveles:

- Mesa electoral.
- Municipio
- Superior a Municipio.

En el contexto de las elecciones en España, las mesas electorales forman parte de la Administración Electoral, junto con las Juntas Electorales. A la Mesa electoral le corresponde presidir el acto de la votación, controlar el desarrollo de la votación y realizar el recuento y el escrutinio.

En términos geográficos cada seccion censal* esta compuesta por al menos una mesa electoral, por lo que para explotar los datos con el objetivo de poder representar los resultados geográficamente debemos agregar a nivel de seccion censal.

En este jupyter notebook cargaremos los datos relativos a las elecciones generales del 23 de julio de 2023 al congreso de los Diputados a nivel de mesa electoral,  los datos relativos a las identificaciones de los partidos politicos y las geometrías de las secciones censales con el objetivo de configurar un fichero único.

Para este fichero único tenemos varias opciones, en primer lugar por la extension del archivo:

- GeoJson: formato estándar abierto diseñado para representar elementos geográficos sencillos, junto con sus atributos no espaciales, basado en JavaScript Object Notation.

- ShapeFile: formato ESRI Shapefile (SHP) es un formato de archivo informático propietario de datos espaciales desarrollado por la compañía ESRI, quien crea y comercializa software para Sistemas de Información Geográfica como Arc/Info o ArcGIS.

Por tipo de dato generado:

- votos_secciones: este será el output generado que dispondrá de la información agregada por seccion censal para todos los partidos así como sus datos geometricos para la representación.
- votos_secciones_simp: mismo concepto que *votos_secciones* pero en este caso los partidos menos representativos* se agregan en la categoría *OTROS*.
- votos_max: Para determinada visualizaciones consideramos interesante generar un fichero ligero que únicamente muestre el partido más votado por seccion censal.




*Se consideran poco representativos y susceptibles de entrar en la categoría "OTROS" todos aquellos partidos que no superene un mínimo  del 1,5% de votos a nivel provincial.


In [51]:
# -*- coding: utf-8 -*-
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

C:\Users\lopez\AppData\Local\Temp\ipykernel_2660\3840362959.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


**Requerimientos**

Especificaremos las librerías requeridas para la ejecución de los procesos:

- pandas
- geopandas
- zipfile
- os
- shutil 

In [52]:
import os
import shutil
import pandas as pd
import geopandas as gpd
import zipfile

**Como guardamos los ficheros geospaciales resultantes**

In [ ]:
save_as_geojson = True
save_as_shapefile = True

<a id='1'></a>

**Datos necesarios para construir los mapas**

Origen de los datos: Ministerio del Interior

https://infoelectoral.interior.gob.es/es/elecciones-celebradas/area-de-descargas/



Serán 3 fuentes las necesarias:

- **PARTIDOS POLITICOS**

- **RESULTADOS ELECTORALES GENERALES**

Secciones censales:

https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout

In [54]:
nb_path= os.getcwd()+'\\data\\'
nb_path = nb_path.replace('\\', '/')
nb_path

'C:/Users/lopez/Documents/GitHub/spanish_elections_results/data/'

In [116]:
out_path = nb_path+'outputs/'
os.makedirs(out_path, exist_ok=True)

In [56]:
# Ruta del archivo zip y nombre del archivo dentro del zip que quieres leer
zip_path = nb_path + '//inputs//02202307_MESA.zip'

# Carpeta temporal para descomprimir
temp_folder = r'data/inputs/temp'

os.makedirs(temp_folder, exist_ok=True)

# Abrir el archivo zip en modo lectura
with zipfile.ZipFile(zip_path, 'r') as archivo_zip:
    # Extraer todo el contenido del archivo zip a la carpeta temporal
    archivo_zip.extractall(path=temp_folder)

# Leer el archivo shapefile con geopandas
parties_path = os.path.join(temp_folder, '03022307.DAT')
parties = pd.read_csv(parties_path, sep='\s{2,}', encoding='ISO-8859-1', header=None, dtype=str)

# Mostrar los primeros 3 registros del GeoDataFrame
parties.head()

C:\Users\lopez\AppData\Local\Temp\ipykernel_2660\3901413921.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  parties = pd.read_csv(parties_path, sep='\s{2,}', encoding='ISO-8859-1', header=None, dtype=str)


,0,1,2
0,02202307000001FO,FRENTE OBRERO,000001000001000001
1,02202307000002PSOE,PARTIDO SOCIALISTA OBRERO ESPAÑOL,000002000002000002
2,02202307000003PUM+J,POR UN MUNDO MÁS JUSTO,000003000003000003
3,02202307000004ALM,ALMERIENSES - REGIONALISTAS PRO ALMERÍA,000004000004000004
4,02202307000005PP,PARTIDO POPULAR,000005000005000005


Tras separar los bloques por 2 o más espacios encontramos en la columna 2 los id relativos a la candidatura. Son 3 códigos: código de la candidatura cabecera de acumulación a nivel provincial (posiciones ), a nivel autonómico y a nivel nacional.


In [57]:
def clean_parties(df):
    df[2] = df[2].astype('str')
    df['id_partido_provincial'] = df[2].str[:6]
    df['id_partido_autonomico'] = df[2].str[6:12]
    df['id_partido_nacional'] = df[2].str[-6:]
    df['partido'] = df[1]
    df = df.drop([0,1,2], axis=1)
    df = df.drop_duplicates()
    return df

In [58]:
# Renombramos los partidos para unificar terminología independientemente de su id_partido_xx
def replacenames_duplicate_parties(df):
    df.loc[df['id_partido_nacional'] == '000010', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_nacional'] == '000002', 'partido'] = 'PSOE'
    df.loc[df['id_partido_nacional'] == '000015', 'partido'] = 'PCTE'
    df.loc[df['id_partido_nacional'] == '000003', 'partido'] = 'POR UN MUNDO MÁS JUSTO'
    df.loc[df['id_partido_nacional'] == '000005', 'partido'] = 'PP'
    df.loc[df['id_partido_nacional'] == '000007', 'partido'] = 'PACMA'
    df.loc[df['id_partido_nacional'] == '000014', 'partido'] = 'ESCAÑOS EN BLANCO PARA DEJAR ESCAÑOS VACÍOS'
    df.loc[df['id_partido_nacional'] == '000023', 'partido'] = 'COALICIÓN EXISTE'
    df.loc[df['id_partido_nacional'] == '000075', 'partido'] = 'PNV'

    df.loc[df['id_partido_autonomico'] == '000030', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000033', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000066', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000021', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000011', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000051', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000067', 'partido'] = 'POR UN MUNDO MÁS JUSTO'

    df.loc[df['id_partido_autonomico'] == '000072', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000029', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000076', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000056', 'partido'] = 'PP'
    df.loc[df['id_partido_autonomico'] == '000064', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000068', 'partido'] = 'PCTE'
    df.loc[df['id_partido_autonomico'] == '000078', 'partido'] = 'PCTE'
    df.loc[df['id_partido_autonomico'] == '000068', 'partido'] = 'PCTE'
    df.loc[df['id_partido_autonomico'] == '000082', 'partido'] = 'SUMAR'

    df = df.drop_duplicates()
    return df

In [59]:
def get_single_id(df):
    df_prov = df[['id_partido_provincial','partido']]
    df_prov = df_prov.rename(columns={'id_partido_provincial': 'id_partido'})

    df_auto = df[['id_partido_autonomico','partido']]
    df_auto = df_auto.rename(columns={'id_partido_autonomico': 'id_partido'})

    df_nac = df[['id_partido_nacional','partido']]
    df_nac = df_nac.rename(columns={'id_partido_nacional': 'id_partido'})

    df_concat = pd.concat([df_prov, df_auto, df_nac])
    df_concat = df_concat.drop_duplicates()
    
    return df_concat

In [60]:
# Transformar pandas df a geopandas df
def df2gdf(df, geometry_field, crs_type):
    gdf = gpd.GeoDataFrame(df, geometry=geometry_field, crs=crs_type)
    return gdf

In [61]:
parties = clean_parties(parties)
parties.head()

,id_partido_provincial,id_partido_autonomico,id_partido_nacional,partido
0,000001,000001,000001,FRENTE OBRERO
1,000002,000002,000002,PARTIDO SOCIALISTA OBRERO ESPAÑOL
2,000003,000003,000003,POR UN MUNDO MÁS JUSTO
3,000004,000004,000004,ALMERIENSES - REGIONALISTAS PRO ALMERÍA
4,000005,000005,000005,PARTIDO POPULAR


In [62]:
parties = replacenames_duplicate_parties(parties)
parties.head()

,id_partido_provincial,id_partido_autonomico,id_partido_nacional,partido
0,000001,000001,000001,FRENTE OBRERO
1,000002,000002,000002,PSOE
2,000003,000003,000003,POR UN MUNDO MÁS JUSTO
3,000004,000004,000004,ALMERIENSES - REGIONALISTAS PRO ALMERÍA
4,000005,000005,000005,PP


In [63]:
parties = get_single_id(parties)
parties.head()

,id_partido,partido
0,000001,FRENTE OBRERO
1,000002,PSOE
2,000003,POR UN MUNDO MÁS JUSTO
3,000004,ALMERIENSES - REGIONALISTAS PRO ALMERÍA
4,000005,PP


**DATA**

In [64]:
data_path = nb_path + '//inputs//temp//10022307.DAT'
data = pd.read_csv(data_path, sep=',', encoding='ISO-8859-1', header=None)

# Mostrar los primeros 3 registros del GeoDataFrame
data.head(3)

,0
0,022023071010400101001 A0000010000001
1,022023071010400101001 A0000020000122
2,022023071010400101001 A0000030000000


In [65]:
data.shape

(647309, 1)

**COMUNIDAD AUTONOMA**

In [66]:
def get_region(df):
    df['comunidad_autonoma'] = df[0].str[9:11]
    return df

In [67]:
data = get_region(data)
data.head(2)

,0,comunidad_autonoma
0,022023071010400101001 A0000010000001,01
1,022023071010400101001 A0000020000122,01


**PROVINCIA**

In [68]:
def get_province(df):
    df['provincia'] = df[0].str[11:13]
    prov_data = {'01': 'Araba/Álava', '02': 'Albacete', '03': 'Alicante/Alacant', '04': 'Almería', '05': 'Ávila', '06': 'Badajoz', '07': 'Balears, Illes', '08': 'Barcelona', '09': 'Burgos', '10': 'Cáceres', '11': 'Cádiz', '12': 'Castellón/Castelló', '13': 'Ciudad Real', '14': 'Córdoba', '15': 'Coruña, A', '16': 'Cuenca', '17': 'Girona', '18': 'Granada', '19': 'Guadalajara', '20': 'Gipuzkoa', '21': 'Huelva', '22': 'Huesca', '23': 'Jaén', '24': 'León', '25': 'Lleida', '26': 'Rioja, La', '27': 'Lugo', '28': 'Madrid', '29': 'Málaga', '30': 'Murcia', '31': 'Navarra', '32': 'Ourense', '33': 'Asturias', '34': 'Palencia', '35': 'Palmas, Las', '36': 'Pontevedra', '37': 'Salamanca', '38': 'Santa Cruz de Tenerife', '39': 'Cantabria', '40': 'Segovia', '41': 'Sevilla', '42': 'Soria', '43': 'Tarragona', '44': 'Teruel', '45': 'Toledo', '46': 'Valencia/València', '47': 'Valladolid', '48': 'Bizkaia', '49': 'Zamora', '50': 'Zaragoza', '51': 'Ceuta', '52': 'Melilla'}
    prov = pd.DataFrame(list(prov_data.items()), columns=['id_prov', 'provincia_nom'])
    df = pd.merge(df, prov, left_on='provincia', right_on='id_prov', how='left')
    df = df.drop(['id_prov'], axis=1)
    return df

In [69]:
data = get_province(data)
data.head(2)

,0,comunidad_autonoma,provincia,provincia_nom
0,022023071010400101001 A0000010000001,01,04,Almería
1,022023071010400101001 A0000020000122,01,04,Almería


**MUNICIPIO**

In [70]:
def get_municipality(df):
    df['municipio'] = data[0].str[11:16]
    return df

In [71]:
data = get_municipality(data)
data.head(2)

,0,comunidad_autonoma,provincia,provincia_nom,municipio
0,022023071010400101001 A0000010000001,01,04,Almería,04001
1,022023071010400101001 A0000020000122,01,04,Almería,04001


**Checks municipio**

022023071 17 03031 01001 U0000020000156

Benidorm:

- 34 10 03 03031 (real INE)
- 34 17 03 03031

El Escorial:

- 34 13 2828054 (real INE)
- 34 12 2828054

Discrepacias con la CCAA

In [72]:
data.shape

(647309, 5)

In [73]:
data[data['municipio'].str.contains('28054')]

,0,comunidad_autonoma,provincia,provincia_nom,municipio
449504,022023071122805401001 A0000010000002,12,28,Madrid,28054
449505,022023071122805401001 A0000020000192,12,28,Madrid,28054
449506,022023071122805401001 A0000030000002,12,28,Madrid,28054
449507,022023071122805401001 A0000050000247,12,28,Madrid,28054
449508,022023071122805401001 A0000060000074,12,28,Madrid,28054
449509,022023071122805401001 A0000070000007,12,28,Madrid,28054
449510,022023071122805401001 A0000090000002,12,28,Madrid,28054
449511,022023071122805401001 A0000100000078,12,28,Madrid,28054
449512,022023071122805401001 A0000150000001,12,28,Madrid,28054
449513,022023071122805401001 A0000190000000,12,28,Madrid,28054


**DISTRITO Y SECCION CENSAL**

In [74]:
def get_census_section(df):
    df['seccion_censal'] = df[0].str[16:21]
    df['seccion_censal'] = df['municipio'] + df['seccion_censal']
    return df

In [75]:
data = get_census_section(data)
data.head(3)

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal
0,022023071010400101001 A0000010000001,01,04,Almería,04001,0400101001
1,022023071010400101001 A0000020000122,01,04,Almería,04001,0400101001
2,022023071010400101001 A0000030000000,01,04,Almería,04001,0400101001


**PARTIDO POLÍTICO**

In [76]:
def get_id_partie(df):
    df['id_partido'] = df[0].str[23:29]
    return df

In [77]:
data = get_id_partie(data)
data.head(3)

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido
0,022023071010400101001 A0000010000001,01,04,Almería,04001,0400101001,000001
1,022023071010400101001 A0000020000122,01,04,Almería,04001,0400101001,000002
2,022023071010400101001 A0000030000000,01,04,Almería,04001,0400101001,000003


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647309 entries, 0 to 647308
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   0                   647309 non-null  object
 1   comunidad_autonoma  647309 non-null  object
 2   provincia           647309 non-null  object
 3   provincia_nom       647022 non-null  object
 4   municipio           647309 non-null  object
 5   seccion_censal      647309 non-null  object
 6   id_partido          647309 non-null  object
dtypes: object(7)
memory usage: 34.6+ MB


**Check número votos de las mesas de una seccion censal:**

In [79]:
data[(data['municipio']=='28054') & (data['seccion_censal']=='2805401002') & (data['id_partido']=='000002')]

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido
449527,022023071122805401002 A0000020000132,12,28,Madrid,28054,2805401002,000002
449538,022023071122805401002 B0000020000146,12,28,Madrid,28054,2805401002,000002
449549,022023071122805401002 C0000020000143,12,28,Madrid,28054,2805401002,000002


**VOTOS**

In [80]:
def get_votes(df):
    df['votos'] = df[0].str[29:36]
    df['votos'] = df['votos'].astype('int')
    return df

In [81]:
data = get_votes(data)
data.head(2)

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos
0,022023071010400101001 A0000010000001,01,04,Almería,04001,0400101001,000001,1
1,022023071010400101001 A0000020000122,01,04,Almería,04001,0400101001,000002,122


In [82]:
data[(data['municipio']=='28054') & (data['seccion_censal']=='2805401002')].groupby(['seccion_censal','id_partido'])['votos'].sum().reset_index(name='numero_de_votos')

,seccion_censal,id_partido,numero_de_votos
0,2805401002,000001,3
1,2805401002,000002,421
2,2805401002,000003,3
3,2805401002,000005,537
4,2805401002,000006,199
5,2805401002,000007,6
6,2805401002,000009,2
7,2805401002,000010,218
8,2805401002,000015,2
9,2805401002,000019,0


**FILTRO CERA (Censo Electoral de Residentes Ausentes) para los agregados provinciales - autonómicos**

In [83]:
data.shape

(647309, 8)

In [84]:
data = data[(data['comunidad_autonoma']!='99')]
data = data[(data['provincia']!='99')]

In [85]:
#data['INE_muni'] = data['municipio'].str[-3:]
#data = data[data['INE_muni']!='999']

In [86]:
data.shape

(647022, 8)

**TABLA SECCIONES CENSALES + ID PARTIDO + VOTOS**

In [87]:
data_secc = data.groupby(['comunidad_autonoma','provincia', 'provincia_nom', 'municipio', 'seccion_censal','id_partido'])['votos'].sum().reset_index(name='votos')
data_secc.head(5)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos
0,01,04,Almería,04001,0400101001,000001,4
1,01,04,Almería,04001,0400101001,000002,294
2,01,04,Almería,04001,0400101001,000003,0
3,01,04,Almería,04001,0400101001,000004,6
4,01,04,Almería,04001,0400101001,000005,262


**UNION CON TABLA DE PARTIDOS**

In [88]:
data_secc = data_secc.merge(parties, on='id_partido', how='left')
data_secc.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO
1,01,04,Almería,04001,0400101001,000002,294,PSOE
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO


In [89]:
data_secc.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO
1,01,04,Almería,04001,0400101001,000002,294,PSOE
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO


**DESCRIPCION PROCESO: OBTENER LOS PARTIDOS REPRESENTATIVOS**

Para determinar aquellos partidos más representativos, con el objetivo de simplificar la información vamos a realizar el siguiente procedimiento:
1. Calculamos los votos por partido a nivel provincial.
2. Calculamos el porcentaje que representan dichos votos.
3. Obtenemos un listado de partidos que superen por provincia un limite de representatividad (1,5%)
4. Creamos un nuevo campo 'partido_simp' que aplica la categoría 'OTROS' a aquellos partidos que no superen el límite establecido.

In [90]:
partidos_provincia = data_secc.groupby(['provincia','partido'])['votos'].sum().reset_index().sort_values(by='provincia', ascending=True)
partidos_provincia.head(2)

,provincia,partido,votos
0,01,ESCAÑOS EN BLANCO PARA DEJAR ESCAÑOS VACÍOS,619
1,01,EUSKAL HERRIA BILDU,32987


**OBTENER PORCENTAJE DE VOTOS POR PARTIDO Y PROVINCIA**

In [91]:
def get_percentaje_vote_prov(df):
    df_totales = df.groupby(['comunidad_autonoma','provincia'])['votos'].sum().reset_index(name='votos').sort_values(by='votos', ascending=False)
    df_totales = df_totales.rename(columns={'votos': 'votos_totales'})
    df_join = pd.merge(df, df_totales, on=['comunidad_autonoma','provincia'], how='left')
    df_join['porcentaje_voto_prov'] = df_join['votos']/df_join['votos_totales']
    df_join = df_join.groupby(['comunidad_autonoma','provincia','partido'])['porcentaje_voto_prov'].sum().reset_index().sort_values(by='porcentaje_voto_prov', ascending=False)
    return df_join

In [92]:
representative_parties = get_percentaje_vote_prov(data_secc)
representative_parties.head(10)

,comunidad_autonoma,provincia,partido,porcentaje_voto_prov
400,11,27,PP,0.505633
411,11,32,PP,0.503246
545,19,52,PP,0.496236
265,08,37,PP,0.472590
500,16,26,PP,0.460634
275,08,40,PP,0.454695
310,08,49,PP,0.450745
216,08,05,PP,0.435340
390,11,15,PP,0.434602
160,06,39,PP,0.424708


**OBTENER LISTA DE PARTIDOS REPRESENTATIVOS**

In [93]:
limit = 0.015
def get_representative_parties(df, limit):
    df = df.groupby(['comunidad_autonoma','provincia','partido'])['porcentaje_voto_prov'].sum().reset_index().sort_values(by='porcentaje_voto_prov', ascending=False)
    df = df[df['porcentaje_voto_prov']>=limit]
    df = df.groupby(['partido']).sum().reset_index().sort_values(by='porcentaje_voto_prov', ascending=False)
    list_parties = df['partido'].tolist()
    return list_parties

In [94]:
representative_parties_list = get_representative_parties(representative_parties, limit)
representative_parties_list

['PP',
 'PSOE',
 'VOX',
 'SUMAR',
 'EUSKAL HERRIA BILDU',
 'PNV',
 'ESQUERRA REPUBLICANA DE CATALUNYA',
 'JUNTS PER CATALUNYA - JUNTS',
 'BLOQUE NACIONALISTA GALEGO',
 'COALICIÓN CANARIA',
 'SORIA ¡YA!',
 'COALICIÓN EXISTE',
 'UNION DEL PUEBLO NAVARRO',
 "CANDIDATURA D'UNITAT POPULAR-PER LA RUPTURA",
 'UNIÓN DEL PUEBLO LEONÉS',
 'NUEVA CANARIAS - BLOQUE CANARISTA',
 'POR ÁVILA',
 'COALICIÓN POR MELILLA',
 'GEROA BAI',
 'JAÉN MERECE MÁS',
 'VAMOS PALENCIA',
 'ZAMORA SÍ']

**AÑADIMOS NUEVO CAMPO QUE SIMPLIFIQUE LAS ETIQUETAS**

In [95]:
def check_partie(partie):
    if partie not in representative_parties_list:
        return 'OTROS'
    else:
        return partie

In [96]:
data_secc['partido_simp'] = data_secc['partido'].apply(lambda x: check_partie(x))
data_secc.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido,partido_simp
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO,OTROS
1,01,04,Almería,04001,0400101001,000002,294,PSOE,PSOE
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO,OTROS


**OBTENEMOS EL PORCENTAJE DE VOTO POR SECCION CENSAL**

In [97]:
def get_percentaje_votee_censussection(df):
    df_totales = df.groupby(['comunidad_autonoma','provincia','municipio','seccion_censal'])['votos'].sum().reset_index(name='votos').sort_values(by='votos', ascending=False)
    df_totales = df_totales.rename(columns={'votos': 'votos_totales'})
    df_join = pd.merge(df, df_totales, on=['comunidad_autonoma','provincia','municipio','seccion_censal'], how='left')
    df_join['porcentaje_voto_secc'] = df_join['votos']/df_join['votos_totales'].round(3)
    return df_join

In [98]:
data_secc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393229 entries, 0 to 393228
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   comunidad_autonoma  393229 non-null  object
 1   provincia           393229 non-null  object
 2   provincia_nom       393229 non-null  object
 3   municipio           393229 non-null  object
 4   seccion_censal      393229 non-null  object
 5   id_partido          393229 non-null  object
 6   votos               393229 non-null  int32 
 7   partido             393229 non-null  object
 8   partido_simp        393229 non-null  object
dtypes: int32(1), object(8)
memory usage: 25.5+ MB


In [99]:
data_census = get_percentaje_votee_censussection(data_secc)
data_census.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido,partido_simp,votos_totales,porcentaje_voto_secc
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO,OTROS,737,0.005427
1,01,04,Almería,04001,0400101001,000002,294,PSOE,PSOE,737,0.398915
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO,OTROS,737,0.000000


**CARGAMOS LAS GEOMETRIAS DE LAS SECCIONES CENSALES**

In [100]:
# Ruta del archivo zip y nombre del archivo dentro del zip que quieres leer
zip_path = nb_path + '//inputs//seccionado_2023.zip'
folder_zip = 'España_Seccionado2023_ETRS89H30/'

# Carpeta temporal para descomprimir
temp_folder = r'data/inputs/temp'

os.makedirs(temp_folder, exist_ok=True)

# Abrir el archivo zip en modo lectura
with zipfile.ZipFile(zip_path, 'r') as archivo_zip:
    # Extraer todo el contenido del archivo zip a la carpeta temporal
    archivo_zip.extractall(path=temp_folder)

# Leer el archivo shapefile con geopandas
shp_path = os.path.join(temp_folder, folder_zip, 'SECC_CE_20230101.shp')
census_gdf = gpd.read_file(shp_path)

# Mostrar los primeros 3 registros del GeoDataFrame
census_gdf.head(3)

# Eliminamos la carpeta temporal
shutil.rmtree(temp_folder)

In [101]:
census_gdf['geo_seccion_censal'] = census_gdf['CUMUN'] + census_gdf['CDIS'] + census_gdf['CSEC']
census_gdf.head(3)

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,CLAU2,NPRO,NCA,CNUT0,CNUT1,CNUT2,CNUT3,NMUN,geometry,geo_seccion_censal
0,0100101001,01001,001,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"MULTIPOLYGON (((539753.044 4743324.668, 539784...",0100101001
1,0100101002,01001,002,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"POLYGON ((539559.740 4745571.157, 539562.677 4...",0100101002
2,0100201001,01002,001,01,002,01,16,0100201,01002,Araba/Álava,País Vasco,ES,2,1,1,Amurrio,"MULTIPOLYGON (((503618.553 4759559.798, 503620...",0100201001


In [102]:
def get_geo_census_section(gdf):
    gdf['geo_seccion_censal'] = gdf['CUMUN'] + gdf['CDIS'] + gdf['CSEC']
    return gdf

In [103]:
census_gdf = get_geo_census_section(census_gdf)
census_gdf.head(3)

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,CLAU2,NPRO,NCA,CNUT0,CNUT1,CNUT2,CNUT3,NMUN,geometry,geo_seccion_censal
0,0100101001,01001,001,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"MULTIPOLYGON (((539753.044 4743324.668, 539784...",0100101001
1,0100101002,01001,002,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"POLYGON ((539559.740 4745571.157, 539562.677 4...",0100101002
2,0100201001,01002,001,01,002,01,16,0100201,01002,Araba/Álava,País Vasco,ES,2,1,1,Amurrio,"MULTIPOLYGON (((503618.553 4759559.798, 503620...",0100201001


In [104]:
def clean_geo_census_section(gdf):
    gdf = gdf.rename(columns={'NCA': 'geo_comunidad_autonoma_nom','NMUN': 'geo_municipio_nom'})
    gdf = gdf[['geo_seccion_censal', 'geo_municipio_nom', 'geo_comunidad_autonoma_nom', 'geometry']]
    return gdf

In [105]:
secc_gdf = clean_geo_census_section(census_gdf)
secc_gdf.head(3)

,geo_seccion_censal,geo_municipio_nom,geo_comunidad_autonoma_nom,geometry
0,0100101001,Alegría-Dulantzi,País Vasco,"MULTIPOLYGON (((539753.044 4743324.668, 539784..."
1,0100101002,Alegría-Dulantzi,País Vasco,"POLYGON ((539559.740 4745571.157, 539562.677 4..."
2,0100201001,Amurrio,País Vasco,"MULTIPOLYGON (((503618.553 4759559.798, 503620..."


**OUTPUTS**

**OUTPUT TODOS LOS PARTIDOS PARA CADA SECCION CENSAL**

In [106]:
all_parties_census = pd.merge(data_census, census_gdf, left_on='seccion_censal', right_on='geo_seccion_censal', how='left')
all_parties_census.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido,partido_simp,votos_totales,...,CLAU2,NPRO,NCA,CNUT0,CNUT1,CNUT2,CNUT3,NMUN,geometry,geo_seccion_censal
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO,OTROS,737,...,04001,Almería,Andalucía,ES,6,1,1,Abla,"POLYGON ((521376.506 4120050.126, 521431.507 4...",0400101001
1,01,04,Almería,04001,0400101001,000002,294,PSOE,PSOE,737,...,04001,Almería,Andalucía,ES,6,1,1,Abla,"POLYGON ((521376.506 4120050.126, 521431.507 4...",0400101001
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO,OTROS,737,...,04001,Almería,Andalucía,ES,6,1,1,Abla,"POLYGON ((521376.506 4120050.126, 521431.507 4...",0400101001


**CHEQUEO Y FILTRO DE LOS RESULTADOS**

In [107]:
print('Número secciones censales con datos de voto pero sin geometría asociada: ' +
      str(all_parties_census[all_parties_census['geometry']==None].shape[0])+' - '+
      str(
          round((all_parties_census[all_parties_census['geometry']==None].shape[0])/(all_parties_census.shape[0]),1))+
      '%'
     )

Número secciones censales con datos de voto pero sin geometría asociada: 1032 - 0.0%


In [108]:
check_geoms_nodata = pd.merge(data_census, census_gdf, left_on='seccion_censal', right_on='geo_seccion_censal', how='outer')
print('Geometrias de secciones censales que no tienen con datos de voto: ' +
      str(check_geoms_nodata[check_geoms_nodata['partido'].isnull()].shape[0])+' - '+
      str(round((check_geoms_nodata[check_geoms_nodata['partido'].isnull()].shape[0])/(check_geoms_nodata.shape[0]),1))+'%'
      )

Geometrias de secciones censales que no tienen con datos de voto: 49 - 0.0%


In [109]:
all_parties_census = all_parties_census[all_parties_census['geometry']!=None]
print('Items válidos a exportar: ' + str(all_parties_census.shape[0]))

Items válidos a exportar: 392197


**GUARDADO**

In [112]:
type(all_parties_census)

pandas.core.frame.DataFrame

In [113]:
all_parties_census = df2gdf(all_parties_census, 'geometry', 'EPSG:25830')
type(all_parties_census)

geopandas.geodataframe.GeoDataFrame

In [117]:
# Guardar el GeoDataFrame como GeoJSON
if save_as_geojson:
    output_file = out_path + 'votos_secciones.geojson'
    all_parties_census.to_file(output_file, driver='GeoJSON')

In [ ]:
if save_as_shapefile:
    output_file = out_path + 'votos_secciones.shp'
    all_parties_census.to_file(output_file, driver="ESRI Shapefile")

**OUTPUT TODOS LOS PARTIDOS PARA CADA SECCION CENSAL SIMPLIFICADOS**

In [ ]:
all_parties_simp_census = all_parties_census.groupby(['comunidad_autonoma','provincia', 'provincia_nom','municipio','seccion_censal',
                                                  'partido_simp'])[['votos','porcentaje_voto_secc']].sum() \
                                                    .reset_index().sort_values(by='votos', ascending=False)
all_parties_simp_census = pd.merge(all_parties_simp_census, census_gdf, left_on='seccion_censal', right_on='geo_seccion_censal', how='left')

all_parties_simp_census.head(2)

In [ ]:
all_parties_simp_census = df2gdf(all_parties_simp_census, 'geometry', 'EPSG:25830')
type(all_parties_simp_census)

In [ ]:
# Guardar el GeoDataFrame como GeoJSON
output_file = out_path + 'votos_secciones_simp.geojson'
all_parties_simp_census.to_file(output_file, driver='GeoJSON')

In [ ]:
output_file = out_path + 'votos_secciones_simp.shp'
all_parties_simp_census.to_file(output_file, driver="ESRI Shapefile")

**OUTPUT DEL PARTIDO MÁS VOTADO**

**ANÁLISIS DIFERENCIAS CON LA PUBLICACION EN EL BOE**

Comenzamos analizando los resultados de un partido de ambito nacional como el PP.
Discrepa el resultado obtenido en los totales:

Boe: 8.160.837

Datos: 8.296.413

Diferencia: +135.576



Analizamos los resultados a nivel provincial y encontramos discrepancias en los totales para Toledo, donde se registran los siguientes resultados:

Boe: 146.533

Datos: 146.813

Diferencia: +280



Si la suma entre provincial solo discrepa en 280, lejos de los mas de 135k de la suma original, hagamos el agragado filtrando las provincias CERA.

Boe: 8.160.837

Datos: 8.161.117

Diferencia: +280



Aplicando este mismo filtro, comparemos otro partido tambien de ambito nacional como el PSOE, en este caso deberemos sumar PSOE + PSC

Boe: 7.821.718 = 6.600.383 + 1.221.335

Datos: 7.821.777

Diferencia: +59

La diferencia ahora es mucho menor entre los resultados obtenidos en los datos y los publicados en el BOE.

Investiguemos los 280 votos de diferencia que hay en el PP en Toledo > No encuentro nada raro, por lo que tal vez debamos de buscar en los datos originales.


In [ ]:
data_secc[(data_secc['partido_simp']=='PP')].groupby(['partido_simp']).sum().reset_index().sort_values(by='votos', ascending=True)

In [ ]:
#Creamos un campo extra temporal para filtrar municipio CERA
data_secc['INE_muni'] = data_secc['municipio'].str[-3:]

In [ ]:
data_secc[(data_secc['partido_simp']=='PP')].groupby(['provincia_nom','partido_simp']).sum().reset_index().sort_values(by='provincia_nom', ascending=True)

In [ ]:
data_secc[(data_secc['partido_simp']=='PP')&(data_secc_parties['provincia']!='99')].groupby(['partido_simp']).sum().reset_index().sort_values(by='votos', ascending=True)

In [ ]:
data_secc[(data_secc['partido_simp']=='PSOE')&(data_secc['provincia']!='99')].groupby(['partido_simp']).sum().reset_index().sort_values(by='votos', ascending=True)

In [ ]:
data_secc[(data_secc['partido_simp']=='PP')&(data_secc['provincia']=='45')&(data_secc['provincia']!='99')].sort_values(by='seccion_censal', ascending=True)

In [ ]:
data_path = 'C://Users//jelopez//Documents//Vodafone//Python//Dev//Jupyter//secciones_censales_politica//data/10022307.DAT'
data = pd.read_csv(data_path, sep=',', encoding='ISO-8859-1', header=None)
data.head()

In [ ]:
data = get_prov(data)
data = data[data['provincia']=='45']
data.head(3)

In [ ]:
data = get_municipio(data)
data.head(3)

In [ ]:
data = get_seccion_censal(data)
data.shape

In [ ]:
data = get_seccion_censal(data)
data.head(3)